In [19]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import config
from derma.dataset import Derma
from derma.architecture import InvertedResidual

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models import MobileNetV2
from torchvision import transforms

torch.cuda.is_available()

True

# Problem parameters

In [20]:
# PARAMS

problem_name = 'Mn_irv00_2_128_raw_weighted' # 'block_setting_classes_size-hairs_weighted-classes'

#dataset_dir = os.path.join(config.DATASET_DIR,'PH2','Clases','')
dataset_dir = os.path.join(config.DATASET_DIR,'ISIC_2020','Train','0128-0128_raw')

CoordAtt = False
Weighted_sampling = True

#lr = 1e-3
lr = 1e-4
n_epoch = 10
batch_size = 32

inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]
# Original setting for mobilenet v2 (https://github.com/pytorch/vision/blob/main/torchvision/models/mobilenetv2.py)

# OTHER PARAMS
labels = [0, 1]
split_ratio = [0.8, 0.1, 0.1] # train, val, test
transform = transforms.Compose([
    transforms.ToTensor(),
#    transforms.Resize((128, 128))
])

criterion = torch.nn.CrossEntropyLoss()
log_dir = os.path.join('log',problem_name) 
model_dir = os.path.join('models',problem_name+'.pt')

# Dataset

In [21]:
# Derma dataset 
dataset = Derma(dataset_dir,labels=labels,transform=transform)

# Train-test splittings
#dataset.shuffle(manual_seed=42) 
train_set, val_set, test_set = dataset.split_rand(split_ratio=split_ratio,manual_seed=42)

In [22]:
# Weighted sampling
if Weighted_sampling:
    from derma.dataset import get_samples_weight
    train_sampler, train_weights = get_samples_weight(train_set)
    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=0, sampler=train_sampler)
    val_sampler, val_weights = get_samples_weight(val_set)
    val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=0, sampler=val_sampler)
    test_sampler, test_weights = get_samples_weight(test_set)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=0, sampler=test_sampler)
else:
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)


# Model

In [23]:
# ISIC2018
if CoordAtt:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)
else:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting) # standard MobileNetV2

# Training

In [24]:
from derma.utils import train, train_val
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
tb_writer = SummaryWriter(log_dir=log_dir)

train_val(model,train_loader,val_loader,optimizer,criterion,tb_writer,n_epoch,model_dir=model_dir)


 10%|█         | 1/10 [01:06<09:57, 66.44s/epoch, tacc=0.8125, tls=0.4244]                 

Epoch 1 
 Training Loss: 0.000555277239112241 
 Validation Loss: 0.018342299788605933
Validation Loss Decreased(inf ---> 1.870915) 	 Saving The Model


 20%|██        | 2/10 [02:13<08:52, 66.59s/epoch, tacc=0.8438, tls=0.3375]

Epoch 2 
 Training Loss: 0.0004777264388736305 
 Validation Loss: 0.004431713445513856
Validation Loss Decreased(1.870915 ---> 0.452035) 	 Saving The Model


 30%|███       | 3/10 [03:19<07:44, 66.43s/epoch, tacc=0.8750, tls=0.2874]

...

Epoch 8 
 Training Loss: 0.00030989932041380403 
 Validation Loss: 0.016475166760238948


 90%|█████████ | 9/10 [09:56<01:06, 66.23s/epoch, tacc=0.9688, tls=0.0974]

Epoch 9 
 Training Loss: 0.0001556613426125978 
 Validation Loss: 0.009794730766146792


100%|██████████| 10/10 [11:02<00:00, 66.29s/epoch, tacc=1.0000, tls=0.1304]

Epoch 10 
 Training Loss: 2.6732904212583864e-05 
 Validation Loss: 0.00034235450713073504
Validation Loss Decreased(0.452035 ---> 0.034920) 	 Saving The Model


In [25]:
torch.save(model.state_dict(), model_dir)

# Test

In [26]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    model.load_state_dict(torch.load(model_dir))
    model.to(device)
else:
    model.load_state_dict(torch.load(model_dir), map_location=torch.device('cpu'))

In [ ]:
def metrics(output: torch.Tensor, target: torch.Tensor,weights=None):
    from derma.metric import Metrics
    import pandas as pd
    acc = Metrics.accuracy(output, target)
    sensitivity, specificity, precission, recall = Metrics.performance(output,target,weights=weights)
    var = [[sensitivity, specificity, precission, recall, acc]]
    columns = ['Sensitivity', 'Specificity', 'Precission', 'Recall', 'Accuracy']
    metrics = pd.DataFrame(var, columns=columns)
    return metrics

def test_experiment(model,dataloader,weights=None):
    from torch.autograd import Variable
#    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = 'cpu'
    model = model.to(device) 
    # set model to evaluation mode
    model.eval()
    # compute metrics over the dataset
    for data_batch, labels_batch in dataloader:
        # fetch the next evaluation batch
        data_batch, labels_batch = Variable(data_batch).to(device), Variable(labels_batch).to(device)
        # compute model output
        output_batch = model(data_batch)
        metrics_batch = metrics(output_batch,labels_batch,weights=weights)
    return metrics_batch

In [28]:
#from derma.experiment import test_experiment

metrics_df = test_experiment(model,test_loader)
metrics_df.info()

c:\Users\mcastro\Documents\MCastro\2_Codigo\DermaModelOptimization\notebooks\..\derma\metric.py:39: RuntimeWarning: invalid value encountered in longlong_scalars
  sensitivity = tp / (tp+fn)
c:\Users\mcastro\Documents\MCastro\2_Codigo\DermaModelOptimization\notebooks\..\derma\metric.py:41: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = tp / (tp+fn)
c:\Users\mcastro\Documents\MCastro\2_Codigo\DermaModelOptimization\notebooks\..\derma\metric.py:39: RuntimeWarning: invalid value encountered in longlong_scalars
  sensitivity = tp / (tp+fn)
c:\Users\mcastro\Documents\MCastro\2_Codigo\DermaModelOptimization\notebooks\..\derma\metric.py:41: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = tp / (tp+fn)
c:\Users\mcastro\Documents\MCastro\2_Codigo\DermaModelOptimization\notebooks\..\derma\metric.py:39: RuntimeWarning: invalid value encountered in longlong_scalars
  sensitivity = tp / (tp+fn)
c:\Users\mcastro\Documents\MCastro\2_Codigo\DermaModelO

ValueError: not enough values to unpack (expected 4, got 1)